## Load Data From .mat Files

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from arctic_objects import *
from arctic_functions import *
import scipy.io as sio
import netCDF4 as nc
import numpy as np
import mat73

KeyboardInterrupt: 

In [ ]:
mitGCM_filepath = '../mitGCM/data/'
NEMO_filepath = '../NEMO/data/'

In [ ]:
# Build the necessary grid data for mitGCM fields
mitGCM_grid = {'lat': None, 'lon': None, 'zcell': None, 'vol': None, 'ocnmsk': None}
for key in mitGCM_grid.keys():
    mitGCM_grid[key] = sio.loadmat(mitGCM_filepath + 'mgrid')['mgrid'][key][0,0]
mitGCM_grid['zcell'] = np.squeeze(mitGCM_grid['zcell'])[:-1]

In [ ]:
#Build the necessary regional logicals for mitGCM fields
mitGCM_reg = {'ARC': None, 'CB': None, 'EB': None, 'NAt': None, 'shelf': None, 'slope': None}
for key in mitGCM_reg.keys():
    mitGCM_reg[key] = sio.loadmat(mitGCM_filepath + 'regional_logicals')['ii_' + key]

In [ ]:
# Make mitGCM kappa background fields
kappa_obs = sio.loadmat(mitGCM_filepath + 'kappa_OBS')['kappa']
mitGCM_kappa_bg = {'CTL': None, 'OBS': None, 'REG': None, 'LO': None, 'HI': None}
for key in mitGCM_kappa_bg.keys():
    mitGCM_kappa_bg[key] = KappaBG(key + ' - ' r'$\kappa_{BG}$', 
                                   mitGCM_grid, kappa_obs, mitGCM_reg, ver=key)

In [ ]:
Make mitGCM vertical diffusivity fields
mitGCM_kappa_t = {'CTL': None, 'OBS': None, 'REG': None, 'LO': None, 'HI': None}
mitGCM_kappa_s = {'CTL': None, 'OBS': None, 'REG': None, 'LO': None, 'HI': None}
for i, key in enumerate(mitGCM_kappa_t.keys()):
    kappa_t = mat73.loadmat(mitGCM_filepath + 'runs.mat')['runs']['kappaT'][i]
    kappa_s = mat73.loadmat(mitGCM_filepath + 'runs.mat')['runs']['kappaS'][i]
    mitGCM_kappa_t[key] = Field(key + r' $\kappa_T$', mitGCM_grid, kappa_t, reg_logic=mitGCM_reg)
    mitGCM_kappa_s[key] = Field(key + r' $\kappa_S$', mitGCM_grid, kappa_s, reg_logic=mitGCM_reg)

In [ ]:
# Build the necessary grid data for NEMO fields
NEMO_grid = {'lat': None, 'lon': None, 'zcell': None, 'vol': None, 'ocnmsk': None}
mesh = nc.Dataset(NEMO_filepath + 'mesh_mask_anha4_201509.nc')
NEMO_grid['lat'] = np.array(mesh.variables['nav_lat'])
NEMO_grid['lon'] = np.array(mesh.variables['nav_lon'])
NEMO_grid['zcell'] = -1*np.array(mesh.variables['nav_lev'])
NEMO_grid['vol'] = np.moveaxis(mesh.variables['e1t'][0]*mesh.variables['e2t'][0]*mesh.variables['e3t'][0], 0, -1)
NEMO_grid['ocnmsk'] = np.moveaxis(mesh.variables['tmask'][0], 0, -1)
NEMO_reg = makeRegions(NEMO_grid)

In [ ]:
# Make NEMO vertical eddy diffusivity fields
NEMO_CTL_path = NEMO_filepath + 'ANHA4-EXH005_2012-2016avg_votkeavt.nc'
NEMO_JSL_path = NEMO_filepath + 'ANHA4-VJT002_2012-2016avg_votkeavt.nc'
NEMO_PZN_path = NEMO_filepath + 'ANHA4-VJT003_2012-2016avg_votkeavt.nc'
NEMO_JSLt_path = NEMO_filepath + 'ANHA4-VJT002_2012-2016avg_votkeavtidal.nc'
NEMO_PZNt_path = NEMO_filepath + 'ANHA4-VJT003_2012-2016avg_votkeavtidal.nc'

NEMO_CTL_data = np.moveaxis(nc.Dataset(NEMO_CTL_path).variables['votkeavtfour'], 0, -1)
NEMO_JSL_data = np.moveaxis(nc.Dataset(NEMO_JSL_path).variables['votkeavtfour'], 0, -1)
NEMO_PZN_data = np.moveaxis(nc.Dataset(NEMO_PZN_path).variables['votkeavtfour'], 0, -1)
NEMO_JSLt_data = np.moveaxis(nc.Dataset(NEMO_JSLt_path).variables['votkeavtidalfour'], 0, -1)
NEMO_PZNt_data = np.moveaxis(nc.Dataset(NEMO_PZNt_path).variables['votkeavtidalfour'], 0, -1)

NEMO_CTL_kappa = Field('Control ', r'$\kappa_{full}$', NEMO_grid, NEMO_CTL_data, reg_logic=NEMO_reg)
NEMO_JSL_kappa = Field('J & SL ', r'$\kappa_{full}$', NEMO_grid, NEMO_JSL_data, reg_logic=NEMO_reg)
NEMO_PZN_kappa = Field('Polzin ', r'$\kappa_{full}$', NEMO_grid, NEMO_PZN_data, reg_logic=NEMO_reg)
NEMO_JSLt_kappa = Field('J & SL ', r'$\kappa_{tidal}$', NEMO_grid, NEMO_JSLt_data, reg_logic=NEMO_reg)
NEMO_PZNt_kappa = Field('Polzin ', r'$\kappa_{tidal}$', NEMO_grid, NEMO_PZNt_data, reg_logic=NEMO_reg)

NEMO_JSL_dKappa = Field('J & SL - Control ', r'$\kappa_{full}$', NEMO_grid, NEMO_JSL_data - NEMO_CTL_data, reg_logic=NEMO_reg)
NEMO_PZN_dKappa = Field('Polzin - Control ', r'$\kappa_{full}$', NEMO_grid, NEMO_PZN_data - NEMO_CTL_data, reg_logic=NEMO_reg)
NEMO_JSLt_dKappa = Field('J & SL', r'$\kappa_{full} - \kappa_{tidal}$', NEMO_grid, NEMO_JSL_data - NEMO_JSLt_data, reg_logic=NEMO_reg)
NEMO_PZNt_dKappa = Field('Polzin', r'$\kappa_{full} - \kappa_{tidal}$', NEMO_grid, NEMO_PZN_data - NEMO_PZNt_data, reg_logic=NEMO_reg)
NEMO_framTransect.visualize_transect(NEMO_JSLt_dKappa).savefig(NEMO_savepath+'FramTransect'+'_JSL_dktidal.png')
NEMO_framTransect.visualize_transect(NEMO_PZNt_dKappa).savefig(NEMO_savepath+'FramTransect'+'_PZN_dktidal.png')

In [ ]:
NEMO_JSLt_dKappa = Field('J & SL', r'$\kappa_{full} - \kappa_{tidal}$', NEMO_grid, NEMO_JSL_data - NEMO_JSLt_data, reg_logic=NEMO_reg)
NEMO_PZNt_dKappa = Field('Polzin', r'$\kappa_{full} - \kappa_{tidal}$', NEMO_grid, NEMO_PZN_data - NEMO_PZNt_data, reg_logic=NEMO_reg)

visualize_average_profiles([NEMO_CTL_kappa, NEMO_JSLt_dKappa, NEMO_PZNt_dKappa]).savefig(NEMO_savepath+'DepthProf'+'_dktidal.png')

NEMO_JSLt_dKappa.visualize_maps().savefig(NEMO_savepath+'DepthMap'+'_JSL_dktidal.png')
NEMO_PZNt_dKappa.visualize_maps().savefig(NEMO_savepath+'DepthMap'+'_PZN_dktidal.png')
NEMO_JSLt_dKappa.visualize_distributions().savefig(NEMO_savepath+'DepthDist'+'_JSL_dktidal.png')
NEMO_PZNt_dKappa.visualize_distributions().savefig(NEMO_savepath+'DepthDist'+'_PZN_dktidal.png')
NEMO_canbarTransect.visualize_transect(NEMO_JSLt_dKappa).savefig(NEMO_savepath+'CanbarTransect'+'_JSL_dktidal.png')
NEMO_canbarTransect.visualize_transect(NEMO_PZNt_dKappa).savefig(NEMO_savepath+'CanbarTransect'+'_PZN_dktidal.png')

In [ ]:
# Make Transects
canbar_start = np.array([-144.3258153, 70.29163412])
canbar_end = np.array([25.05097782, 71.335544347])

fram_start = np.array([-16.626389, 80.676944])
fram_end = np.array([11.748611, 79.158611])

mitGCM_canbarTransect = Transect('CANBAR Transect', mitGCM_grid, canbar_start, canbar_end)
mitGCM_framTransect = Transect('FRAM Transect', mitGCM_grid, fram_start, fram_end)
NEMO_canbarTransect = Transect('CANBAR Transect', NEMO_grid, canbar_start, canbar_end)
NEMO_framTransect = Transect('FRAM Transect', NEMO_grid, fram_start, fram_end)

## Visualize Fields and Transects

In [ ]:
NEMO_savepath = '../NEMO/plots/'

NEMO_CTL_kappa.visualize_regional_profile().savefig(NEMO_savepath+'RegProf'+'_CTL_kfull.png')
NEMO_JSL_kappa.visualize_regional_profile().savefig(NEMO_savepath+'RegProf'+'_JSL_kfull.png')
NEMO_PZN_kappa.visualize_regional_profile().savefig(NEMO_savepath+'RegProf'+'_PZN_kfull.png')
NEMO_JSLt_kappa.visualize_regional_profile().savefig(NEMO_savepath+'RegProf'+'_JSL_ktidal.png')
NEMO_PZNt_kappa.visualize_regional_profile().savefig(NEMO_savepath+'RegProf'+'_PZN_ktidal.png')

visualize_average_profiles([NEMO_CTL_kappa, NEMO_JSL_kappa, NEMO_PZN_kappa]).savefig(NEMO_savepath+'DepthProf'+'_kfull.png')
visualize_average_profiles([NEMO_CTL_kappa, NEMO_JSLt_dKappa, NEMO_PZNt_dKappa]).savefig(NEMO_savepath+'DepthProf'+'_dktidal.png')

NEMO_CTL_kappa.visualize_maps().savefig(NEMO_savepath+'DepthMap'+'_CTL_kfull.png')
NEMO_JSL_kappa.visualize_maps().savefig(NEMO_savepath+'DepthMap'+'_JSL_kfull.png')
NEMO_PZN_kappa.visualize_maps().savefig(NEMO_savepath+'DepthMap'+'_PZN_kfull.png')
NEMO_JSLt_kappa.visualize_maps().savefig(NEMO_savepath+'DepthMap'+'_JSL_ktidal.png')
NEMO_PZNt_kappa.visualize_maps().savefig(NEMO_savepath+'DepthMap'+'_PZN_ktidal.png')
NEMO_JSL_dKappa.visualize_maps(log=False).savefig(NEMO_savepath+'DepthMap'+'_JSL_dkfull.png')
NEMO_PZN_dKappa.visualize_maps(log=False).savefig(NEMO_savepath+'DepthMap'+'_PZN_dkfull.png')
NEMO_JSLt_dKappa.visualize_maps().savefig(NEMO_savepath+'DepthMap'+'_JSL_dktidal.png')
NEMO_PZNt_dKappa.visualize_maps().savefig(NEMO_savepath+'DepthMap'+'_PZN_dktidal.png')

NEMO_CTL_kappa.visualize_distributions().savefig(NEMO_savepath+'DepthDist'+'_CTL_kfull.png')
NEMO_JSL_kappa.visualize_distributions().savefig(NEMO_savepath+'DepthDist'+'_JSL_kfull.png')
NEMO_PZN_kappa.visualize_distributions().savefig(NEMO_savepath+'DepthDist'+'_PZN_kfull.png')
NEMO_JSLt_kappa.visualize_distributions().savefig(NEMO_savepath+'DepthDist'+'_JSL_ktidal.png')
NEMO_PZNt_kappa.visualize_distributions().savefig(NEMO_savepath+'DepthDist'+'_PZN_ktidal.png')
NEMO_JSL_dKappa.visualize_distributions(log=False).savefig(NEMO_savepath+'DepthDist'+'_JSL_dkfull.png')
NEMO_PZN_dKappa.visualize_distributions(log=False).savefig(NEMO_savepath+'DepthDist'+'_PZN_dkfull.png')
NEMO_JSLt_dKappa.visualize_distributions().savefig(NEMO_savepath+'DepthDist'+'_JSL_dktidal.png')
NEMO_PZNt_dKappa.visualize_distributions().savefig(NEMO_savepath+'DepthDist'+'_PZN_dktidal.png')

NEMO_canbarTransect.visualize_transect(NEMO_CTL_kappa).savefig(NEMO_savepath+'CanbarTransect'+'_CTL_kfull.png')
NEMO_canbarTransect.visualize_transect(NEMO_JSL_kappa).savefig(NEMO_savepath+'CanbarTransect'+'_JSL_kfull.png')
NEMO_canbarTransect.visualize_transect(NEMO_PZN_kappa).savefig(NEMO_savepath+'CanbarTransect'+'_PZN_kfull.png')
NEMO_canbarTransect.visualize_transect(NEMO_JSL_dKappa, log=False).savefig(NEMO_savepath+'CanbarTransect'+'_JSL_dkfull.png')
NEMO_canbarTransect.visualize_transect(NEMO_PZN_dKappa, log=False).savefig(NEMO_savepath+'CanbarTransect'+'_PZN_dkfull.png')
NEMO_canbarTransect.visualize_transect(NEMO_JSLt_kappa).savefig(NEMO_savepath+'CanbarTransect'+'_JSL_ktidal.png')
NEMO_canbarTransect.visualize_transect(NEMO_PZNt_kappa).savefig(NEMO_savepath+'CanbarTransect'+'_PZN_ktidal.png')
NEMO_canbarTransect.visualize_transect(NEMO_JSLt_dKappa).savefig(NEMO_savepath+'CanbarTransect'+'_JSL_dktidal.png')
NEMO_canbarTransect.visualize_transect(NEMO_PZNt_dKappa).savefig(NEMO_savepath+'CanbarTransect'+'_PZN_dktidal.png')

NEMO_framTransect.visualize_transect(NEMO_CTL_kappa).savefig(NEMO_savepath+'FramTransect'+'_CTL_kfull.png')
NEMO_framTransect.visualize_transect(NEMO_JSL_kappa).savefig(NEMO_savepath+'FramTransect'+'_JSL_kfull.png')
NEMO_framTransect.visualize_transect(NEMO_PZN_kappa).savefig(NEMO_savepath+'FramTransect'+'_PZN_kfull.png')
NEMO_framTransect.visualize_transect(NEMO_JSL_dKappa, log=False).savefig(NEMO_savepath+'FramTransect'+'_JSL_dkfull.png')
NEMO_framTransect.visualize_transect(NEMO_PZN_dKappa, log=False).savefig(NEMO_savepath+'FramTransect'+'_PZN_dkfull.png')
NEMO_framTransect.visualize_transect(NEMO_JSLt_kappa).savefig(NEMO_savepath+'FramTransect'+'_JSL_ktidal.png')
NEMO_framTransect.visualize_transect(NEMO_PZNt_kappa).savefig(NEMO_savepath+'FramTransect'+'_PZN_ktidal.png')
NEMO_framTransect.visualize_transect(NEMO_JSLt_dKappa).savefig(NEMO_savepath+'FramTransect'+'_JSL_dktidal.png')
NEMO_framTransect.visualize_transect(NEMO_PZNt_dKappa).savefig(NEMO_savepath+'FramTransect'+'_PZN_dktidal.png')

NEMO_canbarTransect.visualize_line().savefig(NEMO_savepath+'TransectMap_canbar.png')
NEMO_framTransect.visualize_line().savefig(NEMO_savepath+'TransectMap_fram.png')

In [ ]:
mitGCM_savepath = '../mitGCM/plots/'

for key in mitGCM_kappa_bg.keys():
    mitGCM_kappa_bg[key].visualize_average_profiles().savefig(mitGCM_savepath+'AvgProf'+'_'+key+'_'+'kbg'+'.png')
    mitGCM_kappa_t[key].visualize_average_profiles().savefig(mitGCM_savepath+'AvgProf'+'_'+key+'_'+'kt'+'.png')
    mitGCM_kappa_s[key].visualize_average_profiles().savefig(mitGCM_savepath+'AvgProf'+'_'+key+'_'+'ks'+'.png')
    
    mitGCM_kappa_bg[key].visualize_distributions().savefig(mitGCM_savepath+'DepthDist'+'_'+key+'_'+'kbg'+'.png')
    mitGCM_kappa_t[key].visualize_distributions().savefig(mitGCM_savepath+'DepthDist'+'_'+key+'_'+'kt'+'.png')
    mitGCM_kappa_s[key].visualize_distributions().savefig(mitGCM_savepath+'DepthDist'+'_'+key+'_'+'ks'+'.png')
    
    mitGCM_kappa_bg[key].visualize_distributions().savefig(mitGCM_savepath+'DepthDist'+'_'+key+'_'+'kbg'+'.png')
    mitGCM_kappa_t[key].visualize_distributions().savefig(mitGCM_savepath+'DepthDist'+'_'+key+'_'+'kt'+'.png')
    mitGCM_kappa_s[key].visualize_distributions().savefig(mitGCM_savepath+'DepthDist'+'_'+key+'_'+'ks'+'.png')
    
    mitGCM_kappa_bg[key].visualize_maps().savefig(mitGCM_savepath+'DepthMap'+'_'+key+'_'+'kbg'+'.png')
    mitGCM_kappa_t[key].visualize_maps().savefig(mitGCM_savepath+'DepthMap'+'_'+key+'_'+'kt'+'.png')
    mitGCM_kappa_s[key].visualize_maps().savefig(mitGCM_savepath+'DepthMap'+'_'+key+'_'+'ks'+'.png')
    
    mitGCM_canbarTransect.visualize_transect(mitGCM_kappa_bg[key]).savefig(
        mitGCM_savepath+'CanbarTransect'+'_'+key+'_'+'kbg'+'.png')
    mitGCM_canbarTransect.visualize_transect(mitGCM_kappa_t[key]).savefig(
        mitGCM_savepath+'CanbarTransect'+'_'+key+'_'+'kt'+'.png')
    mitGCM_canbarTransect.visualize_transect(mitGCM_kappa_s[key]).savefig(
        mitGCM_savepath+'CanbarTransect'+'_'+key+'_'+'ks'+'.png')
    
    mitGCM_framTransect.visualize_transect(mitGCM_kappa_bg[key]).savefig(
        mitGCM_savepath+'FramTransect'+'_'+key+'_'+'kbg'+'.png')
    mitGCM_framTransect.visualize_transect(mitGCM_kappa_t[key]).savefig(
        mitGCM_savepath+'FramTransect'+'_'+key+'_'+'kt'+'.png')
    mitGCM_framTransect.visualize_transect(mitGCM_kappa_s[key]).savefig(
        mitGCM_savepath+'FramTransect'+'_'+key+'_'+'ks'+'.png')

mitGCM_canbarTransect.visualize_line().savefig(mitGCM_savepath+'TransectMap_canbar.png')
mitGCM_framTransect.visualize_line().savefig(mitGCM_savepath+'TransectMap_fram.png')